In [2]:
def calculate_rsi(close, period=14):
    delta = close.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    # Wilder의 이동평균을 사용하여 평균 상승 및 평균 하락 계산
    avg_gain = gain.ewm(alpha=1/period, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [1]:
import os
os.listdir("SP500")

['A.csv',
 'AAPL.csv',
 'ABBV.csv',
 'ABNB.csv',
 'ABT.csv',
 'ACGL.csv',
 'ACN.csv',
 'ADBE.csv',
 'ADI.csv',
 'ADM.csv',
 'ADP.csv',
 'ADSK.csv',
 'AEE.csv',
 'AEP.csv',
 'AES.csv',
 'AFL.csv',
 'AIG.csv',
 'AIZ.csv',
 'AJG.csv',
 'AKAM.csv',
 'ALB.csv',
 'ALGN.csv',
 'ALL.csv',
 'ALLE.csv',
 'AMAT.csv',
 'AMCR.csv',
 'AMD.csv',
 'AME.csv',
 'AMGN.csv',
 'AMP.csv',
 'AMT.csv',
 'AMTM.csv',
 'AMZN.csv',
 'ANET.csv',
 'ANSS.csv',
 'AON.csv',
 'AOS.csv',
 'APA.csv',
 'APD.csv',
 'APH.csv',
 'APTV.csv',
 'ARE.csv',
 'ATO.csv',
 'AVB.csv',
 'AVGO.csv',
 'AVY.csv',
 'AWK.csv',
 'AXON.csv',
 'AXP.csv',
 'AZO.csv',
 'BA.csv',
 'BAC.csv',
 'BALL.csv',
 'BAX.csv',
 'BBY.csv',
 'BDX.csv',
 'BEN.csv',
 'BG.csv',
 'BIIB.csv',
 'BK.csv',
 'BKNG.csv',
 'BKR.csv',
 'BLDR.csv',
 'BLK.csv',
 'BMY.csv',
 'BR.csv',
 'BRO.csv',
 'BSX.csv',
 'BWA.csv',
 'BX.csv',
 'BXP.csv',
 'C.csv',
 'CAG.csv',
 'CAH.csv',
 'CARR.csv',
 'CAT.csv',
 'CB.csv',
 'CBOE.csv',
 'CBRE.csv',
 'CCI.csv',
 'CCL.csv',
 'CDNS.csv',

In [8]:
# 1. Close값이 가장 오랫동안 연속으로 하락한 종목을 찾기!
import pandas as pd
df = pd.read_csv("SP500/AAPL.csv")
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index("Unnamed: 0")

def problem1(df):
    count = 0
    for i in range(1, len(df)):
        if df.iloc[-i]['Close'] < df.iloc[-i-1]['Close']:
            count += 1
        else:
            break
    return count

In [10]:
from tqdm import tqdm
box = []
for i in tqdm(os.listdir("SP500")):
    name = i.split(".")[0]
    df = pd.read_csv("SP500/" + i)
    df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
    df = df.set_index("Unnamed: 0")
    a = problem1(df)
    box.append([name, a])

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:18<00:00, 26.62it/s]


In [13]:
result = pd.DataFrame(box, columns = ['Name', 'Count'])
result.sort_values("Count", ascending = False)
# LYB(9)

,Name,Count
291,LYB,9
166,ERIE,8
393,REGN,6
121,CVS,5
449,TSLA,5
...,...,...
335,NI,0
95,CME,0
94,CMCSA,0
339,NRG,0


In [15]:
df = pd.read_csv("SP500/AAPL.csv")
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index("Unnamed: 0")

def problem1(df):
    max_count = 0
    count = 0
    
    for i in range(len(df) - 1):
        if df.iloc[i]['Close'] > df.iloc[i+1]['Close']:
            count += 1
            if count >= max_count:
                max_count = count
        else:
            count = 0
    
    return max_count

In [16]:
box = []
for i in tqdm(os.listdir("SP500")):
    name = i.split(".")[0]
    df = pd.read_csv("SP500/" + i)
    df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
    df = df.set_index("Unnamed: 0")
    a = problem1(df)
    box.append([name, a])

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [10:57<00:00,  1.31s/it]


In [17]:
result = pd.DataFrame(box, columns = ['Name', 'Count'])
result.sort_values("Count", ascending = False)
# CINF, L, SCHW, TEL (16)

,Name,Count
91,CINF,16
275,L,16
407,SCHW,16
435,TEL,16
340,NSC,15
...,...,...
469,VLTO,6
424,SW,6
415,SOLV,6
201,GEV,4


* 2. 이동평균선과 RSI조건에 맞는 종목

In [22]:
df = pd.read_csv("SP500/AAPL.csv")
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index("Unnamed: 0")

def problem2(df):
    df['RSI'] = calculate_rsi(df['Close'])
    df['MA5'] = df['Close'].rolling(window = 5).mean()
    df['MA20'] = df['Close'].rolling(window = 20).mean()
    df['MA60'] = df['Close'].rolling(window = 60).mean()
    df['MA120'] = df['Close'].rolling(window = 120).mean()
    df = df.dropna()
    
    a = df.iloc[-1]
    if (a["MA120"] < a['MA60'] < a['MA20'] < a['MA5'] < a['Close']) and (a['RSI'] >= 70):
        return True
    else:
        return False

In [24]:
box = []
for i in tqdm(os.listdir("SP500")):
    try:
        name = i.split(".")[0]
        df = pd.read_csv("SP500/" + i)
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
        df = df.set_index("Unnamed: 0")
        a = problem2(df)
        box.append([name, a])
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:16<00:00, 31.21it/s]


In [27]:
result = pd.DataFrame(box, columns = ['Name', 'State'])
result = result[result['State'] == True]
len(result)

12

In [35]:
# 2024년 기준 종가가 밴드 하단을 터치하면 다음날 시가에 매수
# 기다렸다가 밴드의 상단을 터치하면 다음날 시가에 매도하는 전략
# 누적수익률이 가장 높은 종목 (20일, 표준편차 * 2)
df = pd.read_csv("SP500/AAPL.csv")
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index("Unnamed: 0")

def problem3(df):
    df['MA20'] = df['Close'].rolling(window = 20).mean()
    df['STD20'] = df['Close'].rolling(window = 20).std()
    df['UB'] = df['MA20'] + 2 * df['STD20']
    df['LB'] = df['MA20'] - 2 * df['STD20']
    df = df.dropna()
    
    df = df[df.index.year == 2024]
    
    cond1 = df['Close'] <= df['LB']
    cond2 = df['Close'] >= df['UB']
    
    import warnings
    warnings.filterwarnings("ignore")
    
    df['구매신호'] = cond1.shift(1).fillna(False)
    df['판매신호'] = cond2.shift(1).fillna(False)
    
    buy_price = -1
    cum_rate = 1
    
    for i in range(len(df)):
        if (df.iloc[i]['구매신호'] == True) and (buy_price == -1):
            buy_price = df.iloc[i]['Open']
    
        elif (df.iloc[i]['판매신호'] == True) and (buy_price != -1):
            sell_price = df.iloc[i]['Open']
            rate = sell_price / buy_price
            cum_rate *= rate
            buy_price = -1
    
    return cum_rate

In [36]:
box = []
for i in tqdm(os.listdir("SP500")):
    try:
        name = i.split(".")[0]
        df = pd.read_csv("SP500/" + i)
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
        df = df.set_index("Unnamed: 0")
        a = problem3(df)
        box.append([name, a])
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:35<00:00, 13.95it/s]


In [38]:
result = pd.DataFrame(box, columns = ['Name', 'Rate'])
result.sort_values("Rate", ascending = False)

,Name,Rate
374,PLTR,2.277030
8,ADI,1.612491
131,DFS,1.531262
344,NVDA,1.519021
54,BBY,1.511672
...,...,...
204,GL,0.738546
162,EPAM,0.707817
287,LULU,0.608847
148,DXCM,0.547539


* 4번 문제

In [44]:
# RSI값과 상승률을 고려한 종목 찾기
# RSI값이 70이상인 종목 중에서 10일 전 종가 대비 현재 종가가 5% 이상 상승한 종목이 몇개?
df = pd.read_csv("SP500/AAPL.csv")
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df = df.set_index("Unnamed: 0")

def problem4(df):
    df['RSI'] = calculate_rsi(df['Close'])
    df["10day_ago_Close"] = df['Close'].shift(10) # 10일 전 종가
    
    a = df.iloc[-1]
    
    if (a['RSI'] >= 70) and (a['Close'] / a['10day_ago_Close'] - 1 >= 0.05):
        return True
    else:
        return False

In [45]:
box = []
for i in tqdm(os.listdir("SP500")):
    try:
        name = i.split(".")[0]
        df = pd.read_csv("SP500/" + i)
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
        df = df.set_index("Unnamed: 0")
        a = problem4(df)
        box.append([name, a])
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:14<00:00, 35.76it/s]


In [48]:
result = pd.DataFrame(box, columns = ['Name', 'State'])
result = result[result['State'] == True]
len(result)

19

* 5번 문제

In [49]:
box = []
for i in tqdm(os.listdir("SP500")):
    try:
        name = i.split(".")[0]
        df = pd.read_csv("SP500/" + i)
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
        df = df.set_index("Unnamed: 0")

        Q = 0
        for j in range(len(df) - 1):
            if df.iloc[j]['Close'] < df.iloc[j+1]['Close']:
                reward = 1
            else:
                reward = 0
            Q = (reward - Q) / (j+1) + Q

        box.append([name, Q])
            
    
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [11:13<00:00,  1.34s/it]


In [51]:
result = pd.DataFrame(box, columns = ['Name', 'Q'])
result.sort_values("Q", ascending = False)

,Name,Q
469,VLTO,0.554717
84,CEG,0.551873
360,PAYC,0.550019
338,NOW,0.544545
234,HWM,0.543868
...,...,...
458,UDR,0.416896
312,MNST,0.396386
66,BRO,0.383987
232,HUBB,0.333422
